<a href="https://colab.research.google.com/github/delvTom/Data-challenge-MS2A-2025/blob/main/notebooks/02xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BASE =  "/content/drive/MyDrive/datachallenge_data"
DATA = f"{BASE}/data"
OUT = f"{BASE}/outputs/submissions"

import os
import pandas as pd

os.makedirs(OUT, exist_ok=True)

print("DATA:", DATA)
print("OUT :", OUT)
print("Fichiers ->", os.listdir(DATA))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DATA: /content/drive/MyDrive/datachallenge_data/data
OUT : /content/drive/MyDrive/datachallenge_data/outputs/submissions
Fichiers -> ['X_test.csv', 'X_train.csv', 'y_train.csv', 'sample_submission.csv']


In [ ]:

import time, threading, requests, os, IPython
from google.colab import output, drive


try:
    drive.mount('/content/drive')
except Exception:
    pass

js = """
function ConnectButton(){
  console.log("Keep alive: reconnecting...");
  document.querySelector("colab-connect-button").click();
}
setInterval(ConnectButton, 60 * 1000);
"""
IPython.display.display(IPython.display.Javascript(js))
print("Keep-alive activé ✅")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

Keep-alive activé ✅


In [ ]:
X_train = pd.read_csv(f"{DATA}/X_train.csv")
y_train = pd.read_csv(f'{DATA}/y_train.csv')
X_test = pd.read_csv(f"{DATA}/X_test.csv")

print("X_train :", X_train.shape)
print("y_train :", y_train.shape)
print("X_test :", X_test.shape)

X_train : (202933, 14)
y_train : (202933, 24)
X_test : (134673, 14)


In [ ]:

X = X_train.drop(columns=["ID"])
X_test_final = X_test.drop(columns=["ID"])


Y = y_train.drop(columns=["ID"])

print("X :", X.shape)
print("Y :", Y.shape)
print("X_test_final :", X_test_final.shape)



X : (202933, 13)
Y : (202933, 23)
X_test_final : (134673, 13)


In [ ]:
print(X_test_final.head())
print(X_test_final.shape)

   Humidity       M12       M13       M14       M15        M4        M5  \
0  0.869916  0.021914 -0.015737  0.009041  0.015662 -0.139344 -0.041396   
1  0.904594 -0.100771 -0.040901  0.006137  0.013496  0.843534  0.501194   
2  0.688853  0.703585  1.029723  0.785505  0.293766 -0.536002 -0.166262   
3  0.392112  0.648665  0.628007  0.416822  0.065360 -0.309533 -0.398324   
4  0.618233  0.155158  0.285085  0.081453  0.031029 -0.801587 -0.709802   

         M6        M7         R        S1        S2        S3  
0 -0.006051  0.001638  1.008217  1.000922  1.003093  0.998643  
1  0.134963  0.024947  1.010621  0.987593  1.003104  1.009033  
2 -0.029509 -0.015855  1.075560  1.015945  1.038617  1.067383  
3 -0.269763 -0.269454  1.160165  1.084636  1.011883  1.008619  
4 -0.279257 -0.089210  1.051850  1.120134  1.003001  1.007571  
(134673, 13)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from xgboost import XGBRegressor



def weighted_rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_pred = np.clip(y_pred, 0.0, 1.0)
    w = np.where(y_true >= 0.5, 1.2, 1.0)
    mse_per_sample = np.mean(w * (y_pred - y_true)**2, axis=1)
    return float(np.sqrt(np.mean(mse_per_sample)))


weighted_rmse_scorer = make_scorer(lambda yt, yp: -weighted_rmse(yt, yp), greater_is_better=False)


In [ ]:

def clean_features(X_train, X_test, lower_q=0.01, upper_q=0.99, drop_humidity=True):


    X_train_clean = X_train.copy()
    X_test_clean = X_test.copy()

    # === Clipping par quantiles
    for col in X_train_clean.columns:
        if col not in ["ID"]:
            low = X_train_clean[col].quantile(lower_q)
            high = X_train_clean[col].quantile(upper_q)
            X_train_clean[col] = X_train_clean[col].clip(low, high)
            X_test_clean[col] = X_test_clean[col].clip(low, high)

    # === Suppression de la colonne Humidity
    if drop_humidity and "Humidity" in X_train_clean.columns:
        X_train_clean = X_train_clean.drop(columns=["Humidity"])
        X_test_clean = X_test_clean.drop(columns=["Humidity"])

    return X_train_clean, X_test_clean


In [ ]:
X_train_clean, X_test_clean = clean_features(X_train, X_test)

print("Avant :", X_train.shape, X_test.shape)
print("Après :", X_train_clean.shape, X_test_clean.shape)


Avant : (202933, 14) (134673, 14)
Après : (202933, 13) (134673, 13)


In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from xgboost import XGBRegressor
def train_and_evaluate_xgb(
    X_train,
    y_train,
    X_test=None,
    cv_splits: int = 5,
    n_iter_search: int = 20,
    random_state: int = 42
):
    """
    Optimise XGBoost avec RandomizedSearchCV sur Weighted RMSE (métrique challenge),
    puis réentraîne en K-Fold avec les meilleurs hyperparams et produit les prédictions test.

    Args:
        X_train: pd.DataFrame - features train
        y_train: pd.DataFrame - labels train
        X_test: pd.DataFrame - features test (facultatif, sinon None)
        cv_splits: int - nombre de folds pour CV
        n_iter_search: int - nombre d’itérations de random search
        random_state: int - graine

    Returns:
        best_model: modèle MultiOutputRegressor(XGBRegressor) entraîné
        best_score: meilleure Weighted RMSE trouvée
        best_params: hyperparamètres optimaux
        y_pred_test: prédictions sur X_test si fourni (sinon None)
    """

    # ===== 1. Base Model XGB
    base = XGBRegressor(
        tree_method="hist",
        eval_metric="rmse",
        random_state=random_state,
        n_jobs=-1
    )
    model = MultiOutputRegressor(base, n_jobs=-1)

    # ===== 2. Grille de recherche
    param_dist = {
        "estimator__n_estimators": [300, 500, 800],
        "estimator__max_depth": [4, 6, 8, 10],
        "estimator__learning_rate": [0.01, 0.03, 0.05, 0.1],
        "estimator__subsample": [0.6, 0.8, 1.0],
        "estimator__colsample_bytree": [0.6, 0.8, 1.0],
        "estimator__min_child_weight": [1, 5, 10],
        "estimator__reg_lambda": [0.0, 0.5, 1.0, 2.0],
        "estimator__reg_alpha": [0.0, 0.1, 0.5]
    }

    # ===== 3. CV + Random Search
    cv = KFold(n_splits=cv_splits, shuffle=True, random_state=random_state)

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=n_iter_search,
        scoring=weighted_rmse_scorer,   # ⚡️ METRIQUE CHALLENGE
        cv=cv,
        verbose=2,
        n_jobs=-1,
        random_state=random_state
    )

    search.fit(X_train, y_train)

    best_params = search.best_params_
    best_score = -search.best_score_   # repasser en RMSE positif
    print(f"✅ Best CV Weighted RMSE (RandomSearch): {best_score:.6f}")
    print("✅ Best Params:", best_params)

    # ===== 4. Réentraîner un modèle final en K-Fold avec les meilleurs params
    oof_preds = np.zeros_like(y_train, dtype=float)
    test_preds = np.zeros((X_test.shape[0], y_train.shape[1])) if X_test is not None else None

    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        print(f"\n🚀 Training fold {fold+1}/{cv_splits}...")
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Nouveau modèle avec meilleurs params
        base_final = XGBRegressor(
            tree_method="hist",
            eval_metric="rmse",
            random_state=random_state,
            n_jobs=-1,
            **{k.replace("estimator__", ""): v for k, v in best_params.items()}
        )
        model_final = MultiOutputRegressor(base_final, n_jobs=-1)

        model_final.fit(X_tr, y_tr)
        oof_preds[val_idx] = model_final.predict(X_val)

        if X_test is not None:
            test_preds += model_final.predict(X_test) / cv_splits

    # Score final sur le train
    final_score = weighted_rmse(y_train.values, oof_preds)
    print(f"\n🎯 Final CV Weighted RMSE (retrain): {final_score:.6f}")

    # ===== 5. Entraînement global final
    base_final = XGBRegressor(
        tree_method="hist",
        eval_metric="rmse",
        random_state=random_state,
        n_jobs=-1,
        **{k.replace("estimator__", ""): v for k, v in best_params.items()}
    )
    best_model = MultiOutputRegressor(base_final, n_jobs=-1)
    best_model.fit(X_train, y_train)

    return best_model, final_score, best_params, test_preds


In [ ]:
best_model, final_score, best_params, y_pred = train_and_evaluate_xgb(
    X_train_clean,
    y_train,
    X_test_clean,
    cv_splits=5,
    n_iter_search=30


In [ ]:
import pandas as pd


target_cols = [c for c in y_train.columns if c != "ID"]

submission = pd.DataFrame(y_pred, columns=target_cols)
submission.insert(0, "ID", X_test["ID"])

# Sauvegarde
submission.to_csv("submission.csv", index=False)

print("✅ Fichier 'submission.csv' généré avec succès !")
